## Packages required

In [ ]:
using GridapTopOpt, Gridap, Gridap.TensorValues

## Initial material properties

In [ ]:
const E = 30e3        # Young's Modulus
const ν = 0.3         # Poison ratio
const G = E/(2*(1+ν))
const α_mat = 12e-6  #(Thermal expansion coeffiicent)
const κ_mat =  1.0   #(Thermal conductivity)

const Height = 10
const Length = 3*Height

const l = Height/5    # Bending length scale
const N = 0.5         # Micropolar parameter

# const λₘₐₜ = 2*G*ν/(1 -2*ν)  ## For plain strain
# const κₘₐₜ = 2*G*N^2/(1-N^2)
# const μₘₐₜ = G*(1-2*(N^2))/(1-N^2)
# const γₘₐₜ = 4*G*l^2

const λₘₐₜ = E*ν/(1-ν^2)  # Modified λ for plane stress (critical change!)   #2*G*ν/(1 -2*ν)
const κₘₐₜ = 2*G*N^2/(1-N^2)
const μₘₐₜ = G*(1-2*(N^2))/(1-N^2)
const γₘₐₜ = 4*G*l^2

const  T0 = 0.0     # Initial temperature in the domain   
const  TApp = 20.0  # Applied temperature at the boundary
const  fApp = 10    # Mechanical load

In [ ]:
# FE parameters
order = 1                                                        # Finite element order
dom = (0,Length,0,Height)                                        # Bounding domain
nx,ny = (225,75)
el_size = (nx,ny)
model = CartesianDiscreteModel(dom, el_size)                     # Initialize the model
el_Δ = get_el_Δ(model)
shift = 0.05
f_Γ_D_1(x) = (x[1] ≈ 0.0)                                        # Left side
f_Γ_D_2(x) = (x[1] ≈ Length)                                     # Right side                                               
f_Γ_D_3(x) = (x[2] ≈ 0.0) && ((x[1] < Length/2 - Length/40) || (x[1] > Length/2 + Length/40)) # Bottom face
f_Γ_D_4(x) = (x[2] ≈ Height)  # Top face
f_Γ_D_5(x) = (Length/4 - Length/60 <= x[1] <= Length/4 + Length/60) && (Height/2 - Height/50 <= x[2] <= Height/2 + Height/50) # Left source
f_Γ_D_6(x) = (3*Length/4 - Length/60 <= x[1] <= 3*Length/4 + Length/60) && (Height/2 - Height/50 <= x[2] <= Height/2 + Height/50) # Right source
                                                                                
f_Γ_N(x) = (x[2] ≈ 0.0) && (Length/2 - Length/40 <= x[1] <= Length/2 + Length/40 )            # Γ_N indicator function

update_labels!(1,model,f_Γ_D_1,"Gamma_D_1")                                 

update_labels!(3,model,f_Γ_D_2,"Gamma_D_2")

update_labels!(4,model,f_Γ_D_3,"Gamma_D_3")

update_labels!(5,model,f_Γ_D_4,"Gamma_D_4")

update_labels!(6,model,f_Γ_D_5,"Gamma_D_5")

update_labels!(7,model,f_Γ_D_6,"Gamma_D_6")

update_labels!(2,model,f_Γ_N,"Gamma_N")

In [ ]:
# FD parameters
γ = 0.1                                                          # HJ eqn time step coeff
γ_reinit = 0.5                                                   # Reinit. eqn time step coeff
max_steps = 45                                                   # Max steps for advection 
tol = 1e-6                                                       # Reinitialisation tolerance                                   

In [ ]:
I2 = SymTensorValue{2,Float64}(1.0 ,0.0 ,1.0)

E_Matrx = TensorValue(0,1,-1,0)

### New CartesianDiscreteModel

function σ_Tot(ε)
    # Plane stress mechanical stress (λₘₐₜ now uses plane stress version)
    σM = λₘₐₜ*tr(ε)*one(ε) + (2*μₘₐₜ + κₘₐₜ)*ε
    return σM
end

function σ_Temp(T)
    # Plane stress thermal stress: α → α*(1+ν) for plane stress
    ε_th = -α_mat * (1+ν) * T * I2  # Note: (1+ν) factor added!
    σM = λₘₐₜ*tr(ε_th)*one(ε_th) + (2*μₘₐₜ + κₘₐₜ)*ε_th
    return σM
end

function σ_Temp0(T0)
    ε_th = -α_mat * (1+ν) * T0 * I2  # Same adjustment
    σM = λₘₐₜ*tr(ε_th)*one(ε_th) + (2*μₘₐₜ + κₘₐₜ)*ε_th
    return σM
end

# Micropolar and conduction terms remain UNCHANGED
function k_gradTemp(∇)
    return κ_mat * ∇
end

function ε_Skw(∇, θ)
    ∇ᵀ = transpose(∇)
    return 0.5*(∇ᵀ - ∇) - (E_Matrx*θ)
end

function σ_Cmod(ϵ_skew)
    return κₘₐₜ*ϵ_skew
end

function M_mod(∇)
    return γₘₐₜ*∇
end

function Skw(u, θ)
    ∇u = ∇(u)
    ∇ᵀ = transpose(∇u)
    return 0.5*(∇ᵀ - ∇u) - (E_Matrx*θ)
end

In [ ]:
# Problem parameters
g = VectorValue(0,-fApp)                                            # Load applied
vf = 0.5                                                         # Volume fraction constraint
lsf_func = initial_lsf((12/Length),0.1)                 # Initial level set function
iter_mod = 10                                                    # VTK Output modulo                        
path = "./Result_tol&Max_stepMod/TApp-$TApp,fApp-$fApp/Mesh-$nx,$ny/N_$N/ElasticMicropolar_l_$l/Dim_$Length,$Height/E_$E/"      # Output path
mkpath(path)                                                     # Create path

In [ ]:
writevtk(model,path*"FixedExampleII")

## Triangulations and measures
Ω = Triangulation(model)
dΩ = Measure(Ω,2*order)
Γ_N = BoundaryTriangulation(model,tags="Gamma_N")
dΓ_N = Measure(Γ_N,2*order)
# Γ_N_Therm = BoundaryTriangulation(model,tags=["Gamma_D_5","Gamma_D_6"])
# dΓ_N_Therm = Measure(Γ_N_Therm,2*order)
vol_D = sum(∫(1)dΩ)

In [ ]:
## Spaces
reffe = ReferenceFE(lagrangian,VectorValue{2,Float64},order)
reffe_scalar = ReferenceFE(lagrangian,Float64,order)
V = TestFESpace(model,reffe;conformity=:H1,dirichlet_tags=["Gamma_D_1","Gamma_D_2"],dirichlet_masks = [(true, true),(true, true)])
U = TrialFESpace(V,[VectorValue(0.0,0.0),VectorValue(0.0,0.0)])   ## Dispalcement Space

R = TestFESpace(model,reffe_scalar;conformity=:H1,dirichlet_tags=["Gamma_D_1","Gamma_D_2"],dirichlet_masks = [(true),(true)])
S = TrialFESpace(R,[0.0,0.0]) # theta(Micro-Rotation)

Q = TestFESpace(model,reffe_scalar;conformity=:H1,dirichlet_tags=["Gamma_D_1","Gamma_D_2","Gamma_D_3","Gamma_D_4","Gamma_D_5","Gamma_D_6","Gamma_N"],dirichlet_masks = [(true),(true),(true),(true),(true),(true),(true)])
P = TrialFESpace(Q,[T0,T0,TApp,T0,TApp,TApp,TApp]) 

In [ ]:
USP = MultiFieldFESpace([U,S,P])
VRQ = MultiFieldFESpace([V,R,Q])

In [ ]:
V_φ = TestFESpace(model,reffe_scalar)
V_reg = TestFESpace(model,reffe_scalar,dirichlet_tags=["Gamma_N"])
U_reg = TrialFESpace(V_reg,[0.0])

In [ ]:
# Level set and interpolator
φh = interpolate(lsf_func,V_φ)
interp₁ = SmoothErsatzMaterialInterpolation(η = (2)*maximum(get_el_Δ(model)),ϵ = 10^-9)    # η = 2 ×  maximum side length of an element.
I₁,H,DH,ρ = interp₁.I,interp₁.H,interp₁.DH,interp₁.ρ
interp₂ = SmoothErsatzMaterialInterpolation(η = (2)*maximum(get_el_Δ(model)),ϵ = 0.03)    # η = 2 ×  maximum side length of an element.
I₂ = interp₁.I

In [ ]:
writevtk(Ω,path*"initial_lsfFixedII",cellfields=["phi"=>φh,
  "ρ(phi)"=>(ρ ∘ φh),"|nabla(phi)|"=>(norm ∘ ∇(φh))])

In [ ]:
a((u,θ,T),(w,v,z),φ)  = ∫((I₁ ∘ φ)*( (ε(w) ⊙ (σ_Tot∘(ε(u)))) +  (ε(w) ⊙ (σ_Temp∘(T)))
                                  + ((Skw(w,v)) ⊙ (σ_Cmod∘(ε_Skw∘(∇(u),θ))) ) +  ((∇(v))⋅ (M_mod∘(∇(θ)))) 
                                  - (v*((E_Matrx) ⊙ (σ_Cmod∘(ε_Skw∘(∇(u),θ)))))))dΩ + ∫((I₂ ∘ φ)*((∇(z) ⋅ (k_gradTemp∘(∇(T)))) ))dΩ

lm((w,v,z),φ)= ∫((I₁ ∘ φ)*(ε(w) ⊙ (σ_Temp0(T0))))dΩ + ∫(w ⋅ g)dΓ_N 

In [ ]:
state_map = RepeatingAffineFEStateMap(1, a, [lm],USP,VRQ,V_φ)

In [ ]:
evo = FiniteDifferenceEvolver(FirstOrderStencil(2,Float64),model,V_φ;max_steps)
  reinit = FiniteDifferenceReinitialiser(FirstOrderStencil(2,Float64),model,V_φ;tol,γ_reinit)
  ls_evo = LevelSetEvolution(evo,reinit)

In [ ]:
function C_plane_stress(λₘₐₜ,μₘₐₜ,κₘₐₜ)
    λ = λₘₐₜ  # Plane stress λ
    μ = μₘₐₜ
    κ = κₘₐₜ
    return SymFourthOrderTensorValue{2}(
        λ+(2μ+κ), 0.0, λ,
        0.0, μ+(κ/2), 0.0,
        λ, 0.0, λ+(2μ+κ)
    )
end

C2 = C_plane_stress(λₘₐₜ,μₘₐₜ,κₘₐₜ)

In [ ]:
function Cᴴ(r,s,uϕ,φ,dΩ,dΓ_N)
    u_s = uϕ[2s-1]; θ_s = uϕ[2s]; T_s = uϕ[2s+1]
    ∫((I₁ ∘ φ)*(C2 ⊙ (ε(u_s) - α_mat*(T_s-T0)*I2) ⊙ (ε(u_s)- α_mat*(T_s-T0)*I2)))dΩ 
end


J(uϕ,φ) = 1*Cᴴ(1,1,uϕ,φ,dΩ,dΓ_N)
C1(uϕ,φ) = ∫(((ρ ∘ φ) - vf)/vol_D)dΩ;

In [ ]:
DC1(q,uϕ,φ) = ∫(-1/vol_D*q*(DH ∘ φ)*(norm ∘ ∇(φ)))dΩ

In [ ]:
pcfs = PDEConstrainedFunctionals(J,[C1],state_map,analytic_dJ=nothing,analytic_dC=[DC1])

In [ ]:
α = 4max_steps*γ*maximum(get_el_Δ(model))
a_hilb(p,q) = ∫(α^2*∇(p)⋅∇(q) + p*q)dΩ;
vel_ext = VelocityExtension(a_hilb,U_reg,V_reg)

In [ ]:
function mine_al_converged(
  m::AugmentedLagrangian;
  L_tol = 1e-6,
  C_tol = 1e-3,
  window = 10
)
  h = m.history
  it = h.niter
  if it <= window
    return false
  end

  # objective stability
  Li = h[:L,it]
  L_prev = h[:L,it-window:it-1]
  A = abs(Li - mean(L_prev)) / max(abs(Li), 1.0) < L_tol

  # constraint satisfaction
  Ci = h[:C,it]
  B = maximum(abs.(Ci)) < C_tol

  return A && B
end

In [ ]:
# ## Optimiser
optimiser = AugmentedLagrangian(pcfs,ls_evo,vel_ext,φh;maxiter = 250,γ,verbose=true,constraint_names=[:Vol],converged=mine_al_converged)

In [ ]:
for (it,uh,φh) in optimiser
    uv,θv,Tv = uh
    data = ["φ"=>φh,"H(φ)"=>(H ∘ φh),"|∇(φ)|"=>(norm ∘ ∇(φh)),"uv"=>uv,"θv"=>θv,"Tv"=>Tv,"ρ(φ)"=>(ρ ∘ φh)]
    iszero(it % iter_mod) && writevtk(Ω,path*"out$it",cellfields= data) 
    write_history(path*"/historymodified$tol,max_steps-$max_steps.txt",optimiser.history)
end

In [ ]:
it = get_history(optimiser).niter; uh = get_state(pcfs);
uv,θv,Tv = uh
writevtk(Ω,path*"out$it;max_step-$max_steps;tol-$tol",cellfields=["φ"=>φh,"H(φ)"=>(H ∘ φh),"|∇(φ)|"=>(norm ∘ ∇(φh)),"uv"=>uv,"θv"=>θv,"Tv"=>Tv,"ρ(φ)"=>(ρ ∘ φh)])